### 종합실습1
##### Requests 혹은 Selenium 모듈을 이용해 부동산114 모바일 페이지로 접속한 다음, 포항 지역의 아파트 매물 시세 200건을 검색하여 엑셀로 저장해 보세요.
##### 링크 : https://m.r114.com/

### 1. Requests로 크롤링하기

In [12]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

#아래에 코드 작성
html_text = ""
for page_no in range(1, 3):
    url = "https://m.r114.com/Search/m5/search/searchDetailCmd.asp?dqAppendHtml=Y&dqPage="+str(page_no)+"&dqSearchTerm=포항&dqOption=apt&dqSearchType=memul&dqSearchSubType=매매&dqContentSize=100&ranSearchVal=undefined&dqSKString="
    res = requests.get(url, verify = False)
    #response 헤더의 Content-Type에 charset 정보가 없음. 이로 인해 response된 데이터가 ISO-8859-1으로
    #인코딩된 것으로 requests 모듈이 자동판단, res.text 값을 ISO-8859-1로 디코딩해버림
    #하지만 실제로는 UTF-8로 인코딩되어 있으므로 UTF-8로 디코딩해야 한글이 정상 출력됨
    #print(res.encoding) # -> 'ISO-8859-1' 출력됨
    this_text = res.content.decode("UTF-8")
    html_text += this_text

soup = BeautifulSoup(html_text, "html.parser")
items = soup.select("li")
results = []
for item in items:
    this_result_dict = {
        "아파트명" : item.select(".tit")[0].text,
        "지역" : item.select(".info_wrap")[0].select("span")[0].text,
        "크기" : item.select(".info_wrap")[0].select("span")[1].text,
        "매매가" : item.select(".price")[0].text.replace("매매", "")
    }
    results.append(this_result_dict)

df = pd.DataFrame(results)
df.to_excel("../outputs/포항시 아파트 매매가 동향.xlsx", index = False)
display(df)

,아파트명,지역,크기,매매가
0,포항자이,포항시 대잠동,115㎡(35평),"62,000만원"
1,포항자이애서턴,포항시 학잠동,112㎡(34평),"51,900만원"
2,포항자이,포항시 대잠동,99㎡(30평),"46,000만원"
3,풍림아이원,포항시 양덕동,117㎡(35평),"27,400만원"
4,포항상도코아루센트럴하임,포항시 상도동,111㎡(34평),"41,500만원"
...,...,...,...,...
195,우방신천지타운,포항시 두호동,106㎡(32평),"19,500만원"
196,효자웰빙타운SK뷰2차,포항시 효자동,130㎡(39평),"41,500만원"
197,양덕4차삼구트리니엔,포항시 양덕동,111㎡(34평),"36,500만원"
198,포항상도코아루센트럴하임,포항시 상도동,111㎡(34평),"47,000만원"


### 2. Selenium으로 크롤링하기

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

#아래에 코드 작성
driver = webdriver.Chrome("chromedriver.exe")
driver.get("https://m.r114.com/")
driver.find_element_by_css_selector(".search > a").click()
driver.find_element_by_css_selector("#dqSearchTerm").send_keys("포항")
driver.find_element_by_css_selector(".inp_wrap > .btn_search").click()
driver.find_element_by_css_selector("#body_layout > div:nth-child(2) > div > ul > li.n1 > a").click()

for i in range(0, 20): #스크롤바가 최하단으로가면 추가로 데이터를 로드함. 20번 반복하여 총 200개의 데이터 로드
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    
html = driver.page_source

soup = BeautifulSoup(html, "html.parser")
items = soup.select("#resultUI > li")
results = []
for item in items:
    this_result_dict = {
        "아파트명" : item.select(".tit")[0].text,
        "지역" : item.select(".info_wrap")[0].select("span")[0].text,
        "크기" : item.select(".info_wrap")[0].select("span")[1].text,
        "매매가" : item.select(".price")[0].text.replace("매매", "")
    }
    results.append(this_result_dict)

df = pd.DataFrame(results)
df.to_excel("../outputs/포항시 아파트 매매가 동향.xlsx", index = False)
display(df)